# Задача 8
___

## Дан файл системы контроля спорткомплекса. Вам нужно:

* Прочитать файл (поля в файле разделены символом табуляции «\t»).
* Вычислить время, проведённое спортсменом в бассейне (разница между временем входа и выхода из бассейна).
* Рассчитать время, проведённое в комплексе (время между входом и выходом в комплекс).
* Вывести в отдельный файл с логами в случае, если данные о выходе спортсмена отсутствуют, а также в случае обнаружения ошибок или неточностей в записях (отсутствие времени входа, выхода, противоречия в записях).

### Вывод программы

```shell
Атлет 10011 провёл в Pool-2: 225 мин.
Атлет 10013 провёл в Center: 195 мин.
Атлет 10011 провёл в Pool-2: 175 мин.
Атлет 10017 провёл в Pool-2: 152 мин.

logs.log
2023-08-12 20:05:34.082 | DEBUG    | __main__:<module>:24 - Не зафиксировано время входа атлета 10001 в Pool-1
2023-08-12 20:05:34.082 | DEBUG    | __main__:<module>:24 - Не зафиксировано время входа атлета 10001 в Pool-1
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:14 - Не зафиксировано время выхода атлета 10019 из Pool-2
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:24 - Не зафиксировано время входа атлета 10018 в Pool-1
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:24 - Не зафиксировано время входа атлета 10020 в Pool-2
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:26 - Не зафиксировано время выхода атлета 10019 из Pool-2
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:26 - Не зафиксировано время выхода атлета 10011 из Pool-1
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:26 - Не зафиксировано время выхода атлета 10011 из Center
2023-08-12 20:05:34.083 | DEBUG    | __main__:<module>:26 - Не зафиксировано время выхода атлета 10017 из Center
```
___

# Решение

In [8]:
import csv
from datetime import datetime
from collections import defaultdict
from loguru import logger

# Настройка логирования
logger.add('task8.log', level='DEBUG', format='{time} | {level} | {message}', rotation='10 MB')

def parse_time(time_str):
    """Парсинг времени из строки."""
    return datetime.strptime(time_str, "%d/%m/%Y %H:%M:%S")

def process_activity(file_path):
    """Обработка данных из файла."""
    athlete_data = defaultdict(list)
    errors = []

    # Читаем данные из файла
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            try:
                time = parse_time(row['Date'])
                athlete_id = row['Athlete ID']
                location = row['Location']
                event_type = row['Type']

                athlete_data[athlete_id].append((time, location, event_type))
            except Exception as e:
                errors.append(f"Ошибка в строке {row}: {e}")

    # Сортируем события по времени
    for athlete_id in athlete_data:
        athlete_data[athlete_id].sort(key=lambda x: x[0])

    results = []

    # Обрабатываем события
    for athlete_id, events in athlete_data.items():
        location_stack = {}

        for time, location, event_type in events:
            if event_type == 'In':
                location_stack[location] = time
            elif event_type == 'Out':
                if location in location_stack:
                    start_time = location_stack.pop(location)
                    duration = (time - start_time).total_seconds() / 60
                    results.append(f"Атлет {athlete_id} провёл в {location}: {int(duration)} мин.")
                else:
                    logger.debug(f"Не зафиксировано время входа атлета {athlete_id} в {location}")
            else:
                logger.debug(f"Неизвестный тип события: {event_type} у атлета {athlete_id}")

        # Проверяем на оставшиеся записи
        for location in location_stack:
            logger.debug(f"Не зафиксировано время выхода атлета {athlete_id} из {location}")

    # Записываем ошибки
    for error in errors:
        logger.debug(error)

    return results

# Запуск обработки
file_path = 'activity.csv'
output = process_activity(file_path)

# Вывод результатов
for line in output:
    print(line)


2025-01-22 15:53:12.819 | DEBUG    | __main__:process_activity:51 - Не зафиксировано время входа атлета 10012 в Center
2025-01-22 15:53:12.821 | DEBUG    | __main__:process_activity:51 - Не зафиксировано время входа атлета 10001 в Pool-1
2025-01-22 15:53:12.823 | DEBUG    | __main__:process_activity:51 - Не зафиксировано время входа атлета 10001 в Pool-1
2025-01-22 15:53:12.824 | DEBUG    | __main__:process_activity:57 - Не зафиксировано время выхода атлета 10011 из Pool-1
2025-01-22 15:53:12.826 | DEBUG    | __main__:process_activity:57 - Не зафиксировано время выхода атлета 10011 из Center
2025-01-22 15:53:12.827 | DEBUG    | __main__:process_activity:51 - Не зафиксировано время входа атлета 10013 в Center
2025-01-22 15:53:12.829 | DEBUG    | __main__:process_activity:57 - Не зафиксировано время выхода атлета 10019 из Pool-2
2025-01-22 15:53:12.831 | DEBUG    | __main__:process_activity:51 - Не зафиксировано время входа атлета 10018 в Pool-1
2025-01-22 15:53:12.832 | DEBUG    | __mai

Атлет 10011 провёл в Pool-2: 224 мин.
Атлет 10011 провёл в Pool-2: 174 мин.
Атлет 10017 провёл в Pool-2: 151 мин.
